# Ford vs Ferrari project
by Anna Kostyakova

Цель проекта - построить модель классификации изображений автомобилей.

# IMPORT

In [ ]:
# Загружаем ImageDataAugmentor
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

import keras.models as M
import keras.layers as L
import keras.backend as K

from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.efficientnet import EfficientNetB7, EfficientNetB5
from tensorflow.keras.layers import *

from skimage import io

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

# SETUP

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 12  # эпох на обучение
BATCH_SIZE           = 24 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU 
LR                   = 1e-3
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 336 # какого размера подаем изображения в сеть (было 224)
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

RANDOM_SEED = 42

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/" # рабочая директория

# DATA / EDA

In [ ]:
# Импортируем данные

train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")

# Посмотрим, что из себя представляет датасет train
train_df.head()

In [ ]:
train_df.info()

Пропусков не наблюдается.

In [ ]:
# Посмотрим распределение данных.

train_df.Category.value_counts()

In [ ]:
train_df.Category.hist()

В целом, данные распределены более менее равномерно.

In [ ]:
# На всякий случай, удостоверимся в количестве классов.

train_df.Category.nunique()

Посмотрим, что из себя представляют изображения.

In [ ]:
print('Распаковываем картинки')

for data_zip in ['train.zip', 'test.zip']:
   with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
       z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
   im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
   plt.subplot(3,3, index+1)
   plt.imshow(im)
   plt.title('Class: '+str(random_image_cat[index]))
   plt.axis('off')
plt.show()

In [ ]:
# Посмотрим на пример картинки и размер.

image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# AUGMENTATION

В рамках эксперимента было испробовано несколько методов:
- ImageDataGenerator
- ImageDataGenerator с albumentations в виде функции
- ImageDataAugmentor с albumentations

Наилучшие результаты в итоге показал третий вариант, представленный ниже.

In [ ]:
AUGMENTATIONS = A.Compose([
    A.GaussianBlur(p=0.05),
    A.RandomBrightness(limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, 
                       scale_limit=0.01, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=20, 
                       p=.75),
    A.OneOf([
        A.CenterCrop(height=224, width=200),
        A.CenterCrop(height=200, width=224)],
        p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, 
                                                contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, 
                                                contrast_limit=0.1)],
        p=0.5),
    A.HorizontalFlip(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.FancyPCA(alpha=0.1, 
               always_apply=False, 
               p=0.5),
    A.Resize(IMG_SIZE, IMG_SIZE)
])

In [ ]:
train_datagen = ImageDataAugmentor(rescale=1/255,
                        augment=AUGMENTATIONS, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT
                       )


test_datagen = ImageDataAugmentor(
   rescale=1/255,
   validation_split=VAL_SPLIT,
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',     
   target_size=(IMG_SIZE, IMG_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True, seed=RANDOM_SEED,
   subset='training') 

test_generator = test_datagen.flow_from_directory(
    PATH+'train/',
   target_size=(IMG_SIZE, IMG_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True, seed=RANDOM_SEED,
   subset='validation') 


Посмотрим, что у нас получилось.

In [ ]:
def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()

x,y = train_generator.next()
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,9):
   image = x[i]
   plt.subplot(3,3, i+1)
   plt.imshow(image)
plt.show()

In [ ]:
x,y = test_generator.next()
print('Пример картинок из test_generator')
plt.figure(figsize=(12,8))

for i in range(0,9):
   image = x[i]
   plt.subplot(3,3, i+1)
   plt.imshow(image)
plt.show()

# Model

Был проведен эксперимент использования разных базовых моделей, включая Xception, InceptionV3, EfficientNetB5, EfficientNetB7.
Наилучшие результаты показал InceptionV3.

Используем transfer learning & fine-tuning в 4 шага.

## Step 1

In [ ]:
# base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = input_shape)

# base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape = input_shape)

# base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
# Pамораживаем предварительно обученные веса модели, тренируем только верхние слои

base_model.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(L.GlobalAveragePooling2D())
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Проверим статус отдельных слоев

for layer in model.layers:
    print(layer, layer.trainable)

Был проведен эксперимент по подбору оптимизатора. Adam показал наилучшие результаты.

In [ ]:
model.compile(loss="categorical_crossentropy", 
             optimizer=optimizers.Adam(learning_rate=LR), 
             metrics=["accuracy"])

In [ ]:
# Добавим callbacks для сохранения прогресса обучения, для ранней остановки и уменьшения Learning Rate.

checkpoint = ModelCheckpoint('best_model.hdf5', 
                            monitor = 'val_accuracy', 
                            verbose = 1,
                            mode = 'max',
                            save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_accuracy',
                         patience = 5,
                         restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.25,
                             patience=2,
                             min_lr=0.0000001,
                             verbose=1,
                             mode='auto')
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
# Обучаем

history = model.fit_generator(
       train_generator,
       steps_per_epoch = len(train_generator),
       validation_data = test_generator, 
       validation_steps = len(test_generator),
       epochs = EPOCHS,
       callbacks = callbacks_list)

In [ ]:
# Сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)

model.save('../working/model_step_1.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Посмотрим на графики обучения

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## Step 2

In [ ]:
# Посмотрим, сколько слоев в базовой модели

print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
base_model.trainable = True

# Fine-tune с этого слоя
fine_tune_at = len(base_model.layers)//2

# Заморозим все слои перед `fine_tune_at` слоем
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
len(base_model.trainable_variables)

In [ ]:
# Проверим статус отдельных слоев

for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
LR=0.0001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# Обучаем

history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list)

In [ ]:
model.save('../working/model_step_2.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 3

In [ ]:
base_model.trainable = True

# Fine-tune с этого слоя
fine_tune_at = len(base_model.layers)//4

# Заморозим все слои перед `fine_tune_at` слоем
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
LR=0.00001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# Обучаем

history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list)

На этом шаге улучшения показателей не произошло.

In [ ]:
# model.save('../working/model_step_3.hdf5')
# model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим на графики.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## Step 4

Полностью размораживаем веса.

In [ ]:
base_model.trainable = True

In [ ]:
model.summary()

In [ ]:
LR=0.000001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

In [ ]:
# Обучаем

history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list)

In [ ]:
model.save('../working/model_step_4.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# TTA / Submission

Сделаем несколько прогнозов одной картинки по-разному. Окончательный прогноз получим путем усреднения нескольких прогнозов.

In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
sub_test_datagen = ImageDataAugmentor(rescale=1/255,
                        augment=AUGMENTATIONS, 
                        seed=RANDOM_SEED
                       )

test_sub_generator = sub_test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [ ]:
tta_steps = 10
predictions = []

for i in range(tta_steps):
    preds = model.predict(test_sub_generator, verbose=1) 
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1) #несколько категорий
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #перевернем k,v
predictions = [label_map[k] for k in predictions]


In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, 
                          columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
submission.to_csv('submission.csv', index=False)
print('Save submit')

## Итоги

В результате работы над проектом было сделано:
- transfer learning и fine-tuning (обучение головы -> 50% разморозка весов предобученной модели -> 75% разморозка -> 100% разморозка)
- дополнительные функции callback в Keras
- настройка LR и optimizer
- применен способ заполнения пропусков c помощью ImageDataAugmentor с использованием библиотеки аугментации изображений albumentations
- подобраны переменные (размер картинки, батч, количество эпох)
- добавлена Batch Normalization в архитектуре “головы” модели
- добавлена TTA (Test Time Augmentation)
- подобрана архитектура

Экспеименты, не принесшие результатов
- параметры регуляризации полносвязного слоя нейронной сети 
- архитектуры Xception, EfficientNetB5, EfficientNetB7

Xception и InceptionV3 показали очень похожие результаты с разницей в несколько десятых.
EfficientNet, в свою очередь, показал очень скромные результаты, не доходившие до 90%. Даже при уменьшении размера батча часто вылетала ошибка о нехватке памяти. В связи с отсутствием ресурса, с B7 эксперимент так и не удалось завершить до конца.
Модели подгружались через keras.applications, не через установку efficientnet. При попытке второго была ошибка, которую не удалось решить. Другого объяснения, почему у меня модель сработала хуже, чем у других участников соревнования при равных условиях - я не вижу. Если в своем фидбеке вы этот момент мне поясните - буду благодарна.

Самый лучший результат, тем не менее, продемонстрировала модель Xception БЕЗ fine tuning. Но в рамках учебного проекта было решено пожертовать парой тысячных и показать вариант с fine tuning и InceptionV3, так как при fine tuning именно Inception дала лучшие результаты.



